<div>
<img src="../img/EnzymeML.png" height="50"/>
</div>

<div>
<img src="../img/RDM_course_2024_alpha_crop.png" style="float: right" height="150"/>
</div>

# **Create an EnzymeML document** 
## For later use in NMRpy

> created:  2023-11-01  
> modified: 2024-09-24

### *Notebook setup* <a class="anchor" name="setup"></a>

In [ ]:
import sys
import subprocess
import shutil
import importlib

def install_and_import(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed")
    except ImportError:
        print(f"Installing {package}")
        %pip install {package}

# install git via conda if not installed
if not shutil.which("git"):
    print("Installing git via conda")
    subprocess.run(["conda", "install", "-y", "git"])
    if not shutil.which("git"):
        print("git is not installed")
        sys.exit(1)
print("git is installed")

# install required packages
install_and_import("rich")
install_and_import("pyenzyme")

print("🏁 All set! Restart the Notebook once and you are ready to go.")

In [ ]:
from datetime import datetime

import rich

import pyenzyme as pe
import pyenzyme.equations as peq

def print_metadata(data_model) -> None:
    rich.print(f"Current metadata:\n{data_model.model_dump_json(indent=4)}")

---

### **Table of Contents** <a class="anchor" name="table_of_contents"></a>

- [*Notebook setup*](#setup)
- [Prepare EnzymeML document](#enzymeml)
    - [Create an EnzymeML document for the NMR experiment](#i)
    - [Create the NMR tube reaction vessel](#ii)
    - [Create the various reactants and proteins for later reference](#iii)
    - [Prepare the Measurement and MeasurementData objects for later use](#iv)
    - [Create the phosphoglycerate mutase (PGM) reaction and add the respective reactions elements](#v)
    - [Create the enolase (ENO) reaction and add the respective reaction elements](#vi)
    - [Add the reactions to the EnzymeML document](#vii)
- [Serialize the EnzymeML document for transfer to NMRpy](#serialization)
    - [Write the JSON file](#write)
    - [Inspect the created file](#inspect)
- [Disclosure](#disclosure)

---

### **Prepare EnzymeML document** <a class="anchor" name="enzymeml"></a>

#### Create an EnzymeML document for the NMR experiment <a class="anchor" name="i"></a>

In [ ]:
metadata = pe.EnzymeMLDocument(
    name="Test dataset from phosphoglycerate mutase and enolase (PGM-ENO) catalysed conversion of PEP",
    created=datetime.now().isoformat(),
)

#### Create the NMR tube reaction vessel <a class="anchor" name="ii"></a>

In [ ]:
vessel = pe.Vessel(
    id="nmr_tube_10",
    name="Young tube",
    volume=1.0,
    unit=pe.units.predefined.ml,
)
metadata.vessels.append(vessel)
metadata.modified=datetime.now().isoformat()

#### Create the various reactants and proteins for later reference <a class="anchor" name="iii"></a>

In [ ]:
metadata.add_to_small_molecules(
    id="chem01",
    name="3-Phosphoglycerate",
    vessel_id="nmr_tube_10",
    canonical_smiles="C(C(C(=O)O)O)OP(=O)(O)O",
    constant=False,
)

metadata.add_to_small_molecules(
    id="chem02",
    name="2-Phosphoglycerate",
    vessel_id="nmr_tube_10",
    canonical_smiles="C(C(C(=O)O)OP(=O)(O)O)O",
    constant=False,
)

metadata.add_to_small_molecules(
    id="chem03",
    name="Phosphoenolpyruvate",
    vessel_id="nmr_tube_10",
    canonical_smiles="CC(=O)C(=O)OP(=O)(O)O",
    constant=False,
)

metadata.add_to_small_molecules(
    id="chem04",
    name="Phosphate",
    vessel_id="nmr_tube_10",
    canonical_smiles="[O-]P(=O)([O-])[O-]",    
    constant=False,
)

metadata.add_to_small_molecules(
    id="chem05",
    name="Triethylphosphate (internal standard)",
    vessel_id="nmr_tube_10",
    canonical_smiles="CCOP(=O)(OCC)OCC",    
    constant=True,
)

metadata.add_to_proteins(
    id="prot01",
    name="Total lysate protein",
    vessel_id="nmr_tube_10",
    sequence="UNKNOWN",
    constant=True,
)

#### Prepare the Measurement and MeasurementData objects for later use <a class="anchor" name="iv"></a>

In [ ]:
measurement = pe.Measurement(
    id="meas01",
    name="Time-course 31P NMR spectra",
    ph=7.0,
    temperature=298.15,
    unit=pe.units.predefined.kelvin,
)

In [ ]:
for species_type in ["small_molecules", "proteins", "complexes"]:
    for species in getattr(metadata, species_type):
        measurement.add_to_species(
            species_id=species.id,
            init_conc=0.0,
            data_type=pe.DataTypes.CONCENTRATION,
            data_unit=pe.units.predefined.mM,
            time_unit=pe.units.predefined.min,
        )

In [ ]:
concentrations = dict(
    zip(
        ["chem01", "chem02", "chem03", "chem04", "chem05", "prot01"],
        [10.0, 0.0001, 0.0001, 0.0001, 1.0, 0.1418]
    )
)
for conc in concentrations.items():
    for species in measurement.species:
        if species.species_id == conc[0]:
            species.init_conc = conc[1]

In [ ]:
metadata.measurements.append(measurement)

#### Create the phosphoglycerate mutase (PGM) reaction and add the respective reactions elements <a class="anchor" name="v"></a>

In [ ]:
pgm = peq.build_reaction(
    scheme="chem01 -> chem02 + chem04",
    name="Dephosphorylation",
    modifiers=["prot01"]
)

#### Create the enolase (ENO) reaction and add the respective reaction elements <a class="anchor" name="vi"></a>

In [ ]:
eno = peq.build_reaction(
    scheme="chem02 -> chem03 + chem04",
    name="Glycolysis",
    modifiers=["prot01"]
)

#### Add the reactions to the EnzymeML document <a class="anchor" name="vii"></a>

In [ ]:
metadata.reactions += [pgm, eno]
metadata.modified=datetime.now().isoformat()

print_metadata(metadata)

---

### **Serialize the EnzymeML document for transfer to NMRpy** <a class="anchor" name="serialization"></a>

#### Write the JSON file <a class="anchor" name="write"></a>

In [ ]:
pe.write_enzymeml(doc=metadata, path="./data/pgm-eno.json")

#### Inspect the created file <a class="anchor" name="inspect"></a>

In [ ]:
metadata = pe.load_enzymeml(path="./data/pgm-eno.json")
print_metadata(metadata)

---

### **Disclosure** <a class="anchor" name="disclosure"></a>

**Contributions**

If you wish to contribute to or collaborate with EnzymeML, find us on our [EnzymeML GitHub](https://github.com/EnzymeML)!

**MIT License**

Copyright (c) 2024 EnzymeML

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.